# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(units=100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(units=50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(units=1, activation='linear'))

# Compile the model
baseline_model.compile(
    optimizer='sgd', 
    loss='mse', 
    metrics=['mse']
)

# Train the model
baseline_model.fit(
    X_train, 
    y_train, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val)
)

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(
    optimizer='SGD', 
    loss='mse', 
    metrics=['mse']
)

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 
normalized_input_model.fit(
    X_train,  
    y_train, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val)
)

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [9]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [10]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(units=100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(units=50, activation='relu'))
normalized_model.add(layers.Dense(units=1, activation='linear'))

# Compile the model
normalized_model.compile(
    optimizer='sgd', 
    loss='mse', 
    metrics=['mse']
) 

# Train the model
normalized_model.fit(
    X_train, 
    y_train_scaled, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val_scaled)
)

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4692 - mse: 0.4692 - val_loss: 0.2016 - val_mse: 0.2016
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2449 - mse: 0.2449 - val_loss: 0.1585 - val_mse: 0.1585
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1990 - mse: 0.1990 - val_loss: 0.1342 - val_mse: 0.1342
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1724 - mse: 0.1724 - val_loss: 0.1732 - val_mse: 0.1732
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1554 - mse: 0.1554 - val_loss: 0.1238 - val_mse: 0.1238
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1380 - mse: 0.1380 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1266 - mse: 0.1266 - val_loss: 0.1129 - val_mse: 0.1129
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1178 - m

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [11]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 672us/step - loss: 0.0093 - mse: 0.0093


[0.009340654127299786, 0.009340654127299786]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [12]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 773us/step - loss: 0.1125 - mse: 0.1125


[0.11246155202388763, 0.11246155202388763]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [13]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

26353.21161805877

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [14]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(units=100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(
    optimizer='sgd', 
    loss='mse', 
    metrics=['mse']
)

# Train the model
he_model.fit(
    X_train, 
    y_train_scaled, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val_scaled)
)

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4199 - mse: 0.4199 - val_loss: 0.1614 - val_mse: 0.1614
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2301 - mse: 0.2301 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1957 - mse: 0.1957 - val_loss: 0.1334 - val_mse: 0.1334
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1784 - mse: 0.1784 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1625 - mse: 0.1625 - val_loss: 0.1233 - val_mse: 0.1233
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1443 - mse: 0.1443 - val_loss: 0.0977 - val_mse: 0.0977
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1380 - mse: 0.1380 - val_loss: 0.0979 - val_mse: 0.0979
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1297 - m

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 570us/step - loss: 0.0093 - mse: 0.0093


[0.009329301305115223, 0.009329301305115223]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`) 

In [16]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 706us/step - loss: 0.1043 - mse: 0.1043


[0.10430842638015747, 0.10430842638015747]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [17]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(units=100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))


# Add another hidden layer
lecun_model.add(layers.Dense(units=50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(units=1, activation='linear'))

# Compile the model
lecun_model.compile(
    optimizer='sgd', 
    loss='mse', 
    metrics=['mse']
)

# Train the model
lecun_model.fit(
    X_train, 
    y_train_scaled, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val_scaled)
)

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: 0.4392 - mse: 0.4392 - val_loss: 0.1795 - val_mse: 0.1795
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2197 - mse: 0.2197 - val_loss: 0.1742 - val_mse: 0.1742
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1732 - mse: 0.1732 - val_loss: 0.1462 - val_mse: 0.1462
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1548 - mse: 0.1548 - val_loss: 0.1413 - val_mse: 0.1413
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1399 - mse: 0.1399 - val_loss: 0.1520 - val_mse: 0.1520
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1295 - mse: 0.1295 - val_loss: 0.1394 - val_mse: 0.1394
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1169 - mse: 0.1169 - val_loss: 0.1396 - val_mse: 0.1396
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1092 - m

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0074 - mse: 0.0074


[0.007407438941299915, 0.007407438941299915]

Evaluate the model (`lecun_model`) on validate data (`X_val` and `y_val_scaled`) 

In [19]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 749us/step - loss: 0.1773 - mse: 0.1773


[0.1773299276828766, 0.1773299276828766]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [20]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(units=100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(units=50, activation='relu'))
rmsprop_model.add(layers.Dense(units=1, activation='linear'))

# Compile the model
rmsprop_model.compile(
    optimizer='rmsprop', 
    loss='mse', 
    metrics=['mse']
)

# Train the model
rmsprop_model.fit(
    X_train, 
    y_train_scaled, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val_scaled)
)

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.3140 - mse: 0.3140 - val_loss: 0.1393 - val_mse: 0.1393
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1695 - mse: 0.1695 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1409 - mse: 0.1409 - val_loss: 0.1217 - val_mse: 0.1217
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1007 - mse: 0.1007 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0846 - mse: 0.0846 - val_loss: 0.1099 - val_mse: 0.1099
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0627 - mse: 0.0627 - val_loss: 0.2050 - val_mse: 0.2050
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0559 - mse: 0.0559 - val_loss: 0.1295 - val_mse: 0.1295
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0455 - m

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 550us/step - loss: 0.0027 - mse: 0.0027


[0.0026922307442873716, 0.0026922307442873716]

Evaluate the model (`rmsprop_model`) on validation data (`X_val` and `y_val_scaled`) 

In [22]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 623us/step - loss: 0.0989 - mse: 0.0989


[0.098900705575943, 0.098900705575943]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [23]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(units=100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(units=50, activation='relu'))
adam_model.add(layers.Dense(units=1, activation='linear'))

# Compile the model
adam_model.compile(
    optimizer='Adam', 
    loss='mse', 
    metrics=['mse']
)

# Train the model
adam_model.fit(
    X_train, 
    y_train_scaled, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val_scaled)
)

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.3640 - mse: 0.3640 - val_loss: 0.1172 - val_mse: 0.1172
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1521 - mse: 0.1521 - val_loss: 0.0991 - val_mse: 0.0991
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1151 - mse: 0.1151 - val_loss: 0.1025 - val_mse: 0.1025
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0821 - mse: 0.0821 - val_loss: 0.1024 - val_mse: 0.1024
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0681 - mse: 0.0681 - val_loss: 0.1054 - val_mse: 0.1054
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0534 - mse: 0.0534 - val_loss: 0.1197 - val_mse: 0.1197
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0453 - mse: 0.0453 - val_loss: 0.1223 - val_mse: 0.1223
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0365 - m

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 578us/step - loss: 5.7441e-04 - mse: 5.7441e-04


[0.0005744095542468131, 0.0005744095542468131]

Evaluate the model (`adam_model`) on validation data (`X_val` and `y_val_scaled`)

In [25]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 754us/step - loss: 0.0984 - mse: 0.0984


[0.09839403629302979, 0.09839403629302979]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [26]:
# Evaluate the best model on test data

# he_normal kernel_initializer
#    train: loss: 0.0087 - mse: 0.0087
#    val:   loss: 0.1108 - mse: 0.1108

# lecun_normal kernel_initializer
#    train: loss: 0.0084 - mse: 0.0084
#    val:   loss: 0.1454 - mse: 0.1454

# rmsprop optimizer
#    train: loss: 0.0037 - mse: 0.0037
#    val:   loss: 0.0834 - mse: 0.0834

# Adam optimizer
#    train: loss: 0.0063 - mse: 0.0063
#    val:   loss: 0.1061 - mse: 0.1061

# therefore, rmsprop optimizer is the best
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 660us/step - loss: 0.1631 - mse: 0.1631


[0.1630612164735794, 0.1630612164735794]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [27]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

31732.68800217682

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.